In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns

In [2]:
track_folder = '/home/lzhou/Precipitation/TC_Tracks/CMA_Historical_Data/Track_Data/CMABSTdata'
files = os.listdir(track_folder)
files = [x for x in files if 'txt' in x]

In [3]:
first_read = 0
ar = 0
for year in np.arange(2000,2021): 
    filename = 'CH'+str(year)+'BST.txt'
    infile = os.path.join(track_folder,filename)
    os.path.isfile(infile)

    f = open(infile)
    content = f.readlines()
    f.close()

    fr = 0
    #ar = 0
    while fr < len(content):
        # print(content[fr])
        ss = content[fr].split()
        nr = int(ss[2])    #number of rows for this track 
        nsk = fr + 1       #number of rows to skip when read to table
        #print('rows to skip, rows to read: ', nsk, nr)
    
        # read in the track record. 
        # column unit: Time(UTC), IntensityCategory(Chinese standard), LAT/LON(degree in 0.1),
        # PRES(hPa), WND(2-min mean max. sustained wind near the TC center m/s)

        if first_read == 0: 
            aa = pd.read_csv(infile,skiprows=nsk,nrows=nr,sep='\s+',header=None, \
                             names=['Time','IntensityCategory','LAT','LON','PRES','WND'], \
                             dtype={'Time':'int','IntensityCategory':'int','LAT':'float','LON':'float','PRES':'float','WND':'float'})
            first_read = 1
        else:
            dummy = pd.read_csv(infile,skiprows=nsk,nrows=nr,sep='\s+',header=None, \
                                names=['Time','IntensityCategory','LAT','LON','PRES','WND'], \
                                dtype={'Time':'int','IntensityCategory':'int','LAT':'float','LON':'float','PRES':'float','WND':'float'})
            aa = aa.append(dummy,ignore_index=True)
            
        aa.loc[ar:ar+nr,'IntID'] = ss[1]
        aa.loc[ar:ar+nr,'InYearID'] = ss[3]
        aa.loc[ar:ar+nr,'CNID'] = ss[4]
        aa.loc[ar:ar+nr,'Flag'] = ss[5]            # Flag of the last data line: 0=Decay, 1=Move outside of the region, 2=Merge, 3 = Quais-Stationary
        aa.loc[ar:ar+nr,'TimeInterval'] = ss[6]    # Time interval (hour)
        aa.loc[ar:ar+nr,'Name'] = ss[7]            # Typhoon Name
            
        ar = ar + nr
        fr = fr + nr + 1 

# convert data types
#aa.IntID = aa.IntID.astype('int')
aa.InYearID = aa.InYearID.astype('int')
#aa.CNID = aa.CNID.astype('int')
aa.Flag = aa.Flag.astype('int')
aa.TimeInterval = aa.TimeInterval.astype('int')

# correc lat and lon
aa.LAT = aa.LAT/10.
aa.LON = aa.LON/10.

# convert intergers to datetime format
aa['Time'] = pd.to_datetime(aa['Time'],format='%Y%m%d%H')

In [4]:
aa.Time

0       2000-05-05 00:00:00
1       2000-05-05 06:00:00
2       2000-05-05 12:00:00
3       2000-05-05 18:00:00
4       2000-05-06 00:00:00
                ...        
17373   2020-12-24 06:00:00
17374   2020-12-24 12:00:00
17375   2020-12-24 18:00:00
17376   2020-12-25 00:00:00
17377   2020-12-25 00:00:00
Name: Time, Length: 17378, dtype: datetime64[ns]

In [11]:
aa[aa.Name=='Chanthu']

,Time,IntensityCategory,LAT,LON,PRES,WND,IntID,InYearID,CNID,Flag,TimeInterval,Name
3360,2004-06-07 18:00:00,0,8.6,132.2,1008.0,10.0,0000,8,0405,0,6,Chanthu
3361,2004-06-08 00:00:00,0,9.0,130.5,1008.0,10.0,0000,8,0405,0,6,Chanthu
3362,2004-06-08 06:00:00,1,9.3,128.6,1006.0,12.0,0000,8,0405,0,6,Chanthu
3363,2004-06-08 12:00:00,1,9.5,127.5,1006.0,12.0,0000,8,0405,0,6,Chanthu
3364,2004-06-08 18:00:00,1,10.1,126.4,1006.0,12.0,0000,8,0405,0,6,Chanthu
3365,2004-06-09 00:00:00,1,10.7,125.0,1004.0,12.0,0000,8,0405,0,6,Chanthu
3366,2004-06-09 06:00:00,1,10.9,123.8,1004.0,12.0,0000,8,0405,0,6,Chanthu
3367,2004-06-09 12:00:00,1,11.0,122.4,1004.0,12.0,0000,8,0405,0,6,Chanthu
3368,2004-06-09 18:00:00,1,11.1,121.0,1002.0,12.0,0000,8,0405,0,6,Chanthu
3369,2004-06-10 00:00:00,1,11.4,120.0,1002.0,12.0,0000,8,0405,0,6,Chanthu
